In [1]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from model import MLP
from training_defs import create_dataloader, get_loss_optimizer, SpectraDataset

In [2]:
csv_path = "data/simpler_data_rwc.csv"
df = pd.read_csv(csv_path)

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)/len(df)*100:.2f}%")
print(f"Validation: {len(val_df)/len(df)*100:.2f}%")
print(f"Test: {len(test_df)/len(df)*100:.2f}%")

Train: 69.99%
Validation: 14.97%
Test: 15.03%


In [3]:
train_loader = create_dataloader(train_df)
val_loader   = create_dataloader(val_df, shuffle=False)
test_loader  = create_dataloader(test_df, shuffle=False)

In [4]:
input_dim = len([c for c in train_df.columns if c.isdigit()])

mlp = MLP(input_dim=input_dim, output_dim=3)

loss_fn, optimizer = get_loss_optimizer(mlp, lr=1e-3)

In [6]:
epochs = 20
print("Training started...")
for epoch in range(epochs):
    print(f"Starting epoch {epoch+1}/{epochs}...")
    mlp.train()
    total_loss = 0.0

    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        preds = mlp(X)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if (i+1) % 10 == 0:
            print(f"Loss after batch {i+1}: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} finished - Avg Training Loss: {avg_loss:.4f}")

    mlp.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for X_val, y_val in val_loader:
            val_preds = mlp(X_val)
            val_loss = loss_fn(val_preds, y_val)
            total_val_loss += val_loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"📊 Epoch [{epoch+1}/{epochs}] - Validation Loss: {avg_val_loss:.4f}\n")

print("Training finished!")

Training started...
Starting epoch 1/20...
Loss after batch 10: 0.0460
Loss after batch 20: 0.0622
Loss after batch 30: 0.0610
Epoch 1 finished - Avg Training Loss: 0.0702
📊 Epoch [1/20] - Validation Loss: 0.0526

Starting epoch 2/20...
Loss after batch 10: 0.0303
Loss after batch 20: 0.0343
Loss after batch 30: 0.0302
Epoch 2 finished - Avg Training Loss: 0.0337
📊 Epoch [2/20] - Validation Loss: 0.0363

Starting epoch 3/20...
Loss after batch 10: 0.0304
Loss after batch 20: 0.0244
Loss after batch 30: 0.0212
Epoch 3 finished - Avg Training Loss: 0.0256
📊 Epoch [3/20] - Validation Loss: 0.0311

Starting epoch 4/20...
Loss after batch 10: 0.0170
Loss after batch 20: 0.0247
Loss after batch 30: 0.0189
Epoch 4 finished - Avg Training Loss: 0.0218
📊 Epoch [4/20] - Validation Loss: 0.0264

Starting epoch 5/20...
Loss after batch 10: 0.0251
Loss after batch 20: 0.0222
Loss after batch 30: 0.0175
Epoch 5 finished - Avg Training Loss: 0.0178
📊 Epoch [5/20] - Validation Loss: 0.0243

Starting e

In [9]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [10]:
mlp.eval()

all_preds, all_targets = [], []

with torch.no_grad():  
    for X, y in test_loader:  
        preds = mlp(X)
        all_preds.append(preds.numpy())
        all_targets.append(y.numpy())

all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

# compute metrics
r2 = r2_score(all_targets, all_preds, multioutput="uniform_average")
rmse = np.sqrt(mean_squared_error(all_targets, all_preds))

print(f"Validation R²: {r2:.4f}")
print(f"Validation RMSE: {rmse:.4f}")

Validation R²: 0.8563
Validation RMSE: 0.1163
